In [ ]:
!pip install -q gradio google-generativeai chromadb langchain-community pypdf
!npm install -g localtunnel

code="""

import os
import gradio as gr
import google.generativeai as genai
import chromadb
from chromadb.utils.embedding_functions import EmbeddingFunction
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.api_core import retry

# Set Gemini API key
GEMINI_API_KEY = os.getenv("GENERATIVE_AI_API_KEY")
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Gemini Embedding Function for Chroma
class GeminiEmbeddingFunction(EmbeddingFunction):
    def __init__(self, document_mode=True):
        self.document_mode = document_mode

    def __call__(self, input):
        task_type = "retrieval_document" if self.document_mode else "retrieval_query"
        response = genai.embed_content(
            model="models/text-embedding-004",
            content=input,
            task_type=task_type,
            request_options={"retry": retry.Retry(predicate=retry.if_transient_error)},
        )
        return response["embedding"]

# Load and chunk PDF
def process_pdf(file_path):
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = [doc.page_content for doc in splitter.split_documents(pages)]
    return chunks

# Store chunks into Chroma
def load_into_chroma(chunks):
    embed_fn = GeminiEmbeddingFunction(document_mode=True)
    chroma_client = chromadb.Client()
    collection = chroma_client.get_or_create_collection("gemini_rag", embedding_function=embed_fn)
    ids = [str(i) for i in range(len(chunks))]
    collection.add(documents=chunks, ids=ids)
    return collection, embed_fn

# Gemini answer using retrieved chunks
def query_gemini(question, collection, embed_fn):
    embed_fn.document_mode = False
    results = collection.query(query_texts=[question], n_results=3)
    context = "\\n\\n".join(results["documents"][0])

    model = genai.GenerativeModel("gemini-1.5-flash")
    prompt = f\"\"\"You are a helpful assistant. Use the context below to answer the question.

Context:
{context}

Question: {question}
Answer:\"\"\"
    response = model.generate_content(prompt)
    return response.text, results["documents"][0]

# State for Gradio interface
uploaded_chunks = []
collection = None
embed_fn = None

# Upload and process PDF
def handle_upload(file):
    global uploaded_chunks, collection, embed_fn
    uploaded_chunks = process_pdf(file.name)
    collection, embed_fn = load_into_chroma(uploaded_chunks)
    return f"✅ Indexed {len(uploaded_chunks)} text chunks."

# Chat function
def rag_chat(message, history):
    if not collection:
        return "⚠️ Please upload a PDF first."
    answer, sources = query_gemini(message, collection, embed_fn)
    return answer

# Gradio Interface
with gr.Blocks() as app:
    gr.Markdown("## 📄 Gemini RAG PDF Chatbot")
    gr.Markdown("Upload a PDF and ask questions. Gemini 1.5 Flash will answer using the document.")

    with gr.Row():
        file_input = gr.File(label="📤 Upload PDF", file_types=[".pdf"])
        file_output = gr.Textbox(label="Status")

    file_input.change(fn=handle_upload, inputs=file_input, outputs=file_output)

    chatbot_ui = gr.ChatInterface(
        fn=rag_chat,
        title="🤖 Ask Your PDF",
        theme="soft",
        # retry_btn="🔄 Retry",
        # clear_btn="🧹 Clear Chat",
    )
    chatbot_ui.render()

# Run the app
if __name__ == "__main__":
    app.launch(share=True)

"""
with open("chatbot_gradio_rag.py", "w") as f:
    f.write(code)

import os
os.environ["GEMINI_API_KEY"] = "GENERATIVE_AI_API_KEY"


# !python chatbot_gradio_rag.py & npx localtunnel --port 7860
!!python chatbot_gradio_rag.py

